# 使用注意力机制的LSTM的机器翻译


## 环境设置


In [1]:
import paddle
import paddle.nn.functional as F
import re
import numpy as np

print(paddle.__version__)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


2.0.2


## 构建双语句对的数据结构

接下来通过处理文本文件，将双语句对读入到python的数据结构中。这里做了如下的处理。

- 对于日文，按照分词切分/直接按照字符切分。
- 对于中文，为了简便起见，未做分词，按照字做了切分。
- 为了后续的程序运行的更快，限制句子长度，得到了一个较小的数据集。

In [2]:
MAX_LEN = 20

In [3]:
import pandas as pd
data=pd.read_excel("data/jpn-chn-fenci.xlsx")
data

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xlrd/xlsx.py:39: DeprecationWarning: defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.
  try: import defusedxml.cElementTree as ET


,Unnamed: 0,jpnnum,jpn,chnnum,chn,jpn-fenci,chn-fenci
0,0,1297,きみにちょっとしたものをもってきたよ。,343792,我给你带了点东西。,"['きみ', 'に', 'ちょっと', 'し', 'た', 'もの', 'を', 'もっ',...","['我', '给', '你', '带', '了', '点', '东西']"
1,1,4702,何かしてみましょう。,2431762,做点儿什么尝试下吧。,"['何', 'か', 'し', 'て', 'み', 'ましょう']","['做', '点儿', '什么', '尝试', '下', '吧']"
2,2,4703,私は眠らなければなりません。,2,我该去睡觉了。,"['私', 'は', '眠ら', 'なけれ', 'ば', 'なり', 'ませ', 'ん']","['我', '该', '去', '睡觉', '了']"
3,3,4703,私は眠らなければなりません。,2431761,我必须睡了。,"['私', 'は', '眠ら', 'なけれ', 'ば', 'なり', 'ませ', 'ん']","['我', '必须', '睡', '了']"
4,4,4704,何してるの？,3,你在干什麼啊？,"['何', 'し', 'てる', 'の']","['你', '在', '干什麼', '啊']"
...,...,...,...,...,...,...,...
12969,12969,182546,急がなければ汽車に遅れますよ。,11294953,快点，不然要赶不上火车了。,"['急が', 'なけれ', 'ば', '汽車', 'に', '遅れ', 'ます', 'よ']","['快点', '，', '不然', '要', '赶不上', '火车', '了']"
12970,12970,1529507,理由は知っているけど、君に教えることはできないよ。,11294957,我知道原因，但我不能告诉你。,"['理由', 'は', '知っ', 'て', 'いる', 'けど', '、', '君', '...","['我', '知道', '原因', '，', '但', '我', '不能', '告诉', '你']"
12971,12971,1073492,彼女は無関心を装った。,11294959,她假装不在意。,"['彼女', 'は', '無', '関心', 'を', '装っ', 'た']","['她', '假装', '不在意']"
12972,12972,83070,母はサラダに塩を加えるのを忘れた。,3669927,我妈妈忘了往沙拉里加盐了。,"['母', 'は', 'サラダ', 'に', '塩', 'を', '加える', 'の', '...","['我', '妈妈', '忘', '了', '往', '沙拉', '里加', '盐', '了']"


In [4]:
pairs=[]
for i in range(len(data)):
    pairs.append((eval(str(data['jpn-fenci'][i])),eval(str(data['chn-fenci'][i]))))

In [5]:

# create a smaller dataset to make the demo process faster
filtered_pairs = []

for x in pairs:
    if len(x[0]) < MAX_LEN and len(x[1]) < MAX_LEN:#and x[0][0] in ('i', 'you', 'he', 'she', 'we', 'they'):
        filtered_pairs.append(x)
           
print(len(filtered_pairs))
for x in filtered_pairs[:10]: print(x) 

12661
(['きみ', 'に', 'ちょっと', 'し', 'た', 'もの', 'を', 'もっ', 'て', 'き', 'た', 'よ'], ['我', '给', '你', '带', '了', '点', '东西'])
(['何', 'か', 'し', 'て', 'み', 'ましょう'], ['做', '点儿', '什么', '尝试', '下', '吧'])
(['私', 'は', '眠ら', 'なけれ', 'ば', 'なり', 'ませ', 'ん'], ['我', '该', '去', '睡觉', '了'])
(['私', 'は', '眠ら', 'なけれ', 'ば', 'なり', 'ませ', 'ん'], ['我', '必须', '睡', '了'])
(['何', 'し', 'てる', 'の'], ['你', '在', '干什麼', '啊'])
(['何', 'し', 'てる', 'の'], ['你', '做', '什麼'])
(['今日', 'は', '６', '月', '１８', '日', 'で', '、', 'ムー', 'リエル', 'の', '誕生', '日', 'です'], ['今天', '是', '６', '月', '１', '８', '号', '，', '也', '是', 'Muiriel', '的', '生日'])
(['お', '誕生', '日', 'お', 'めでとう', 'ムー', 'リエル'], ['生日快乐', '，', 'Muiriel'])
(['ムー', 'リエル', 'は', '２０', '歳', 'に', 'なり', 'まし', 'た'], ['Muiriel', '现在', '20', '岁', '了'])
(['パスワード', 'は', '「', 'Muiriel', '」', 'です'], ['密码', '是', '"', 'Muiriel', '"'])


##  创建词表

接下来我们分别创建中英文的词表，这两份词表会用来将日文和中文的句子转换为词的ID构成的序列。词表中还加入了如下三个特殊的词：
- `<pad>`: 用来对较短的句子进行填充。
- `<bos>`: "begin of sentence"， 表示句子的开始的特殊词。
- `<eos>`: "end of sentence"， 表示句子的结束的特殊词。

Note: 在实际的任务中，可能还需要通过`<unk>`（或者`<oov>`）特殊词来表示未在词表中出现的词。

In [6]:
en_vocab = {}
cn_vocab = {}

# create special token for pad, begin of sentence, end of sentence
en_vocab['<pad>'], en_vocab['<bos>'], en_vocab['<eos>'] = 0, 1, 2
cn_vocab['<pad>'], cn_vocab['<bos>'], cn_vocab['<eos>'] = 0, 1, 2

en_idx, cn_idx = 3, 3
for en, cn in filtered_pairs:
    for w in en: 
        if w not in en_vocab: 
            en_vocab[w] = en_idx
            en_idx += 1
    for w in cn:  
        if w not in cn_vocab: 
            cn_vocab[w] = cn_idx
            cn_idx += 1

print(len(list(en_vocab)))
print(len(list(cn_vocab)))

7479
11308


## 创建padding过的数据集

接下来根据词表，将会创建一份实际的用于训练的用numpy array组织起来的数据集。
- 所有的句子都通过`<pad>`补充成为了长度相同的句子。
- 对于日文句子（源语言），将其反转了过来，这会带来更好的翻译的效果。
- 所创建的`padded_cn_label_sents`是训练过程中的预测的目标，即，每个中文的当前词去预测下一个词是什么词。


In [7]:
padded_en_sents = []
padded_cn_sents = []
padded_cn_label_sents = []
for en, cn in filtered_pairs:
    # reverse source sentence
    padded_en_sent = en + ['<eos>'] + ['<pad>'] * (MAX_LEN - len(en))
    padded_en_sent.reverse()
    padded_cn_sent = ['<bos>'] + cn + ['<eos>'] + ['<pad>'] * (MAX_LEN - len(cn))
    padded_cn_label_sent = cn + ['<eos>'] + ['<pad>'] * (MAX_LEN - len(cn) + 1) 

    padded_en_sents.append([en_vocab[w] for w in padded_en_sent])
    padded_cn_sents.append([cn_vocab[w] for w in padded_cn_sent])
    padded_cn_label_sents.append([cn_vocab[w] for w in padded_cn_label_sent])

train_en_sents = np.array(padded_en_sents)
train_cn_sents = np.array(padded_cn_sents)
train_cn_label_sents = np.array(padded_cn_label_sents)

print(train_en_sents.shape)
print(train_cn_sents.shape)
print(train_cn_label_sents.shape)

(12661, 21)
(12661, 22)
(12661, 22)


## 创建网络

创建一个Encoder-AttentionDecoder架构的模型结构用来完成机器翻译任务。
首先将设置一些必要的网络结构中用到的参数。

In [8]:
embedding_size = 128
hidden_size = 256
num_encoder_lstm_layers = 1
en_vocab_size = len(list(en_vocab))
cn_vocab_size = len(list(cn_vocab))
epochs = 20
batch_size = 16

## Encoder部分

在编码器的部分，通过查找完Embedding之后接一个LSTM的方式构建一个对源语言编码的网络。飞桨的RNN系列的API，除了LSTM之外，还提供了SimleRNN, GRU供使用，同时，还可以使用反向RNN，双向RNN，多层RNN等形式。也可以通过`dropout`参数设置是否对多层RNN的中间层进行`dropout`处理，来防止过拟合。

除了使用序列到序列的RNN操作之外，也可以通过SimpleRNN, GRUCell, LSTMCell等API更灵活的创建单步的RNN计算，甚至通过继承RNNCellBase来实现自己的RNN计算单元。

In [9]:
# encoder: simply learn representation of source sentence
class Encoder(paddle.nn.Layer):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = paddle.nn.Embedding(en_vocab_size, embedding_size,)
        self.lstm = paddle.nn.LSTM(input_size=embedding_size, 
                                   hidden_size=hidden_size, 
                                   num_layers=num_encoder_lstm_layers)

    def forward(self, x):
        x = self.emb(x)
        x, (_, _) = self.lstm(x)
        return x

## AttentionDecoder部分

在解码器部分，通过一个带有注意力机制的LSTM来完成解码。

- 单步的LSTM：在解码器的实现的部分，同样使用LSTM，与Encoder部分不同的是，下面的代码，每次只让LSTM往前计算一次。整体的recurrent部分，是在训练循环内完成的。
- 注意力机制：这里使用了一个由两个Linear组成的网络来完成注意力机制的计算，它用来计算出目标语言在每次翻译一个词的时候，需要对源语言当中的每个词需要赋予多少的权重。

In [10]:
# only move one step of LSTM, 
# the recurrent loop is implemented inside training loop
class AttentionDecoder(paddle.nn.Layer):
    def __init__(self):
        super(AttentionDecoder, self).__init__()
        self.emb = paddle.nn.Embedding(cn_vocab_size, embedding_size)
        self.lstm = paddle.nn.LSTM(input_size=embedding_size + hidden_size, 
                                   hidden_size=hidden_size)

        # for computing attention weights
        self.attention_linear1 = paddle.nn.Linear(hidden_size * 2, hidden_size)
        self.attention_linear2 = paddle.nn.Linear(hidden_size, 1)
        
        # for computing output logits
        self.outlinear =paddle.nn.Linear(hidden_size, cn_vocab_size)

    def forward(self, x, previous_hidden, previous_cell, encoder_outputs):
        x = self.emb(x)
        
        attention_inputs = paddle.concat((encoder_outputs, 
                                      paddle.tile(previous_hidden, repeat_times=[1, MAX_LEN+1, 1])),
                                      axis=-1
                                     )

        attention_hidden = self.attention_linear1(attention_inputs)
        attention_hidden = F.tanh(attention_hidden)
        attention_logits = self.attention_linear2(attention_hidden)
        attention_logits = paddle.squeeze(attention_logits)

        attention_weights = F.softmax(attention_logits)        
        attention_weights = paddle.expand_as(paddle.unsqueeze(attention_weights, -1), 
                                             encoder_outputs)

        context_vector = paddle.multiply(encoder_outputs, attention_weights)               
        context_vector = paddle.sum(context_vector, 1)
        context_vector = paddle.unsqueeze(context_vector, 1)
        
        lstm_input = paddle.concat((x, context_vector), axis=-1)

        # LSTM requirement to previous hidden/state: 
        # (number_of_layers * direction, batch, hidden)
        previous_hidden = paddle.transpose(previous_hidden, [1, 0, 2])
        previous_cell = paddle.transpose(previous_cell, [1, 0, 2])
        
        x, (hidden, cell) = self.lstm(lstm_input, (previous_hidden, previous_cell))
        
        # change the return to (batch, number_of_layers * direction, hidden)
        hidden = paddle.transpose(hidden, [1, 0, 2])
        cell = paddle.transpose(cell, [1, 0, 2])

        output = self.outlinear(hidden)
        output = paddle.squeeze(output)
        return output, (hidden, cell)

## 训练模型

接下来开始训练模型。

- 在每个epoch开始之前，对训练数据进行了随机打乱。
- 通过多次调用`atten_decoder`，在这里实现了解码时的recurrent循环。
- `teacher forcing`策略: 在每次解码下一个词时，给定了训练数据当中的真实词作为了预测下一个词时的输入。

In [11]:
encoder = Encoder()
atten_decoder = AttentionDecoder()

opt = paddle.optimizer.Adam(learning_rate=0.001, 
                            parameters=encoder.parameters()+atten_decoder.parameters())

for epoch in range(epochs):
    print("epoch:{}".format(epoch))

    # shuffle training data
    perm = np.random.permutation(len(train_en_sents))
    train_en_sents_shuffled = train_en_sents[perm]
    train_cn_sents_shuffled = train_cn_sents[perm]
    train_cn_label_sents_shuffled = train_cn_label_sents[perm]

    for iteration in range(train_en_sents_shuffled.shape[0] // batch_size):
        x_data = train_en_sents_shuffled[(batch_size*iteration):(batch_size*(iteration+1))]
        sent = paddle.to_tensor(x_data)
        en_repr = encoder(sent)

        x_cn_data = train_cn_sents_shuffled[(batch_size*iteration):(batch_size*(iteration+1))]
        x_cn_label_data = train_cn_label_sents_shuffled[(batch_size*iteration):(batch_size*(iteration+1))]

        # shape: (batch,  num_layer(=1 here) * num_of_direction(=1 here), hidden_size)
        hidden = paddle.zeros([batch_size, 1, hidden_size])
        cell = paddle.zeros([batch_size, 1, hidden_size])

        loss = paddle.zeros([1])
        # the decoder recurrent loop mentioned above
        for i in range(MAX_LEN + 2):
            cn_word = paddle.to_tensor(x_cn_data[:,i:i+1])
            cn_word_label = paddle.to_tensor(x_cn_label_data[:,i])

            logits, (hidden, cell) = atten_decoder(cn_word, hidden, cell, en_repr)
            step_loss = F.cross_entropy(logits, cn_word_label)
            loss += step_loss

        loss = loss / (MAX_LEN + 2)
        if(iteration % 200 == 0):
            print("iter {}, loss:{}".format(iteration, loss.numpy()))

        loss.backward()
        opt.step()
        opt.clear_grad()

W1216 09:50:03.417253   144 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W1216 09:50:03.422962   144 device_context.cc:372] device: 0, cuDNN Version: 7.6.


epoch:0
iter 0, loss:[9.330698]
iter 200, loss:[1.635001]


## 使用模型进行机器翻译

完成上面的模型训练之后，我们可以得到一个能够从日文翻译成中文的机器翻译模型。接下来我们通过一个greedy search来实现使用该模型完成实际的机器翻译。

In [18]:
encoder.eval()
atten_decoder.eval()

num_of_exampels_to_evaluate = 10

indices = np.random.choice(len(train_en_sents),  num_of_exampels_to_evaluate, replace=False)
x_data = train_en_sents[indices]
sent = paddle.to_tensor(x_data)
en_repr = encoder(sent)

word = np.array(
    [[cn_vocab['<bos>']]] * num_of_exampels_to_evaluate
)
word = paddle.to_tensor(word)

hidden = paddle.zeros([num_of_exampels_to_evaluate, 1, hidden_size])
cell = paddle.zeros([num_of_exampels_to_evaluate, 1, hidden_size])

decoded_sent = []
for i in range(MAX_LEN + 2):
    logits, (hidden, cell) = atten_decoder(word, hidden, cell, en_repr)
    word = paddle.argmax(logits, axis=1)
    decoded_sent.append(word.numpy())
    word = paddle.unsqueeze(word, axis=-1)
    
results = np.stack(decoded_sent, axis=1)
for i in range(num_of_exampels_to_evaluate):
    en_input = " ".join(filtered_pairs[indices[i]][0])
    ground_truth_translate = "".join(filtered_pairs[indices[i]][1])
    model_translate = ""
    for k in results[i]:
        w = list(cn_vocab)[k]
        if w != '<pad>' and w != '<eos>':
            model_translate += w
    print(en_input)
    print("true: {}".format(ground_truth_translate))
    print("pred: {}".format(model_translate))